<a href="https://colab.research.google.com/github/Davide-polimi/Projects/blob/main/Generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIBRARY SETUP

In [ ]:
# Uninstall preinstalled TensorFlow
!pip uninstall -y tensorflow tensorflow-text tensorflow-decision-forests tf-keras

# Installa le versioni desiderate (già fatto)
!pip install tensorflow==2.14.0 \
             numpy==1.25.2 \
             protobuf==4.25.3 \
             jax==0.4.20 \
             flax==0.7.2 \
             jaxlib==0.4.20 \
             optax==0.1.7 \
             scipy==1.11.4 --force-reinstall



Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0
  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached jax-0.4.20-py3-none-any.whl.metadata (23 kB)
  Using cached flax-0.7.2-py3-none-any.whl.metadata (10.0 kB)
  Using cached jaxlib-0.4.20-cp311-cp311-manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached optax-0.1.7-py3-none-any.whl.metadata (13 kB)
  Using cached scipy-1.11.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (87

In [ ]:
import tensorflow as tf
import numpy as np
import scipy
import jax
import flax
import jaxlib
import optax
import scipy
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

print(tf.__version__)
print(np.__version__)
print(scipy.__version__)
print(jax.__version__)
print(flax.__version__)
print(jaxlib.__version__)
print(optax.__version__)
print(scipy.__version__)
from IPython.display import display, Markdown,Image, clear_output
from google.colab import output
import tempfile


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
def plot_plate_3d(dimensions):
    length = dimensions["length_mm"]
    width = dimensions["width_mm"]
    thickness = dimensions["thickness_mm"]

    # Coordinate bounds centered around origin
    x0, x1 = -length / 2, length / 2
    y0, y1 = -width / 2, width / 2
    z0, z1 = -thickness / 2, thickness / 2

    # Define 8 corners of the plate (centered at origin)
    corners = np.array([
        [x0, y0, z0],
        [x1, y0, z0],
        [x1, y1, z0],
        [x0, y1, z0],
        [x0, y0, z1],
        [x1, y0, z1],
        [x1, y1, z1],
        [x0, y1, z1]
    ])

    # Define faces
    faces = [
        [corners[0], corners[1], corners[2], corners[3]],  # bottom
        [corners[4], corners[5], corners[6], corners[7]],  # top
        [corners[0], corners[1], corners[5], corners[4]],  # front
        [corners[1], corners[2], corners[6], corners[5]],  # right
        [corners[2], corners[3], corners[7], corners[6]],  # back
        [corners[3], corners[0], corners[4], corners[7]]   # left
    ]

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')

    for face in faces:
        ax.add_collection3d(
            Poly3DCollection([face], facecolors='gray', edgecolors='k', alpha=0.3)
        )

    # Axes adjustment
    ax.set_box_aspect([length, width, thickness])
    ax.view_init(elev=30, azim=45)
    ax.set_axis_off()

    # Add reference axes centered at (0,0,0)
    arrow_len = max(length, width, thickness) * 0.6
    ax.quiver(0, 0, 0, arrow_len, 0, 0, color='blue', arrow_length_ratio=0.05)
    ax.text(arrow_len * 1.02, 0, 0, 'X', color='blue')
    ax.quiver(0, 0, 0, 0, arrow_len, 0, color='blue', arrow_length_ratio=0.05)
    ax.text(0, arrow_len * 1.02, 0, 'Y', color='blue')
    ax.quiver(0, 0, 0, 0, 0, arrow_len, color='blue', arrow_length_ratio=0.05)
    ax.text(0, 0, arrow_len, 'Z', color='blue')

    plt.tight_layout()
    plt.show()
    time.sleep(0.3)


dimensions = {
    "length_mm": 120.0,
    "width_mm": 80.0,
    "thickness_mm": 10.0
}
plot_plate_3d(dimensions)

In [ ]:
def plot_cylinder_3d(dimensions):
    radius = dimensions["diameter_mm"] / 2
    height = dimensions["height_mm"]
    n_points = 100
    theta = np.linspace(0, 2 * np.pi, n_points)

    # Coordinates for top and bottom circles
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    z_bottom = np.zeros_like(x)
    z_top = np.ones_like(x) * height

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Side faces
    for i in range(n_points - 1):
        verts = [[
            [x[i], y[i], 0],
            [x[i + 1], y[i + 1], 0],
            [x[i + 1], y[i + 1], height],
            [x[i], y[i], height]
        ]]
        ax.add_collection3d(Poly3DCollection(verts, facecolors='gray', edgecolors='none', alpha=0.3))

    # Closing the loop (last segment)
    verts = [[
        [x[-1], y[-1], 0],
        [x[0], y[0], 0],
        [x[0], y[0], height],
        [x[-1], y[-1], height]
    ]]
    ax.add_collection3d(Poly3DCollection(verts, facecolors='gray', edgecolors='none', alpha=0.3))

    # Top and bottom surfaces
    # Bottom face — clean ring (no radial lines)
    bottom_ring = [[x[i], y[i], 0] for i in range(n_points)]
    ax.add_collection3d(Poly3DCollection([bottom_ring], facecolors='gray', edgecolors='k', alpha=0.3, closed=True))

    # Top face — clean ring (no radial lines)
    top_ring = [[x[i], y[i], height] for i in range(n_points)]
    ax.add_collection3d(Poly3DCollection([top_ring], facecolors='gray', edgecolors='k', alpha=0.3, closed=True))

    # Add X, Y, Z blue axes (arrows)
    arrow_color = 'blue'
    arrow_length = dimensions["diameter_mm"] * 1.2  # 120% of cylinder diameter

    # Z axis (vertical)
    ax.quiver(0, 0, 0, 0, 0, arrow_length, color=arrow_color, arrow_length_ratio=0.05)
    ax.text(0, 0, arrow_length * 1.02, 'Z', color=arrow_color)

    # X axis (horizontal, right)
    ax.quiver(0, 0, 0, arrow_length, 0, 0, color=arrow_color, arrow_length_ratio=0.05)
    ax.text(arrow_length * 1.02, 0, 0, 'X', color=arrow_color)

    # Y axis (horizontal, forward)
    ax.quiver(0, 0, 0, 0, arrow_length, 0, color=arrow_color, arrow_length_ratio=0.05)
    ax.text(0, arrow_length * 1.02, 0, 'Y', color=arrow_color)

    # Set aspect ratio and view
    ax.set_box_aspect([1, 1, height / radius])
    ax.view_init(elev=30, azim=45)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    time.sleep(0.3)


dimensions = {"diameter_mm": 60.0, "height_mm": 100.0}
plot_cylinder_3d(dimensions)

In [ ]:
def plot_gear_3d(dimensions):
    num_teeth = dimensions["number_of_teeth"]
    outer_radius = dimensions["outer_diameter"] / 2
    root_radius = dimensions["root_diameter"] / 2
    top_width = dimensions["tooth_top_width"]
    base_width = dimensions["tooth_base_width"]
    tooth_height = dimensions["tooth_height"]
    thickness = dimensions["gear_thickness"]
    clearance_angle_deg = dimensions["tooth_clearance_angle"]

    angle_step = 2 * np.pi / num_teeth

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for i in range(num_teeth):
        angle = i * angle_step

        r_base = root_radius
        r_tip = outer_radius
        center_base = np.array([r_base * np.cos(angle), r_base * np.sin(angle)])
        center_tip = np.array([r_tip * np.cos(angle), r_tip * np.sin(angle)])

        tangent = np.array([-np.sin(angle), np.cos(angle)])
        tangent /= np.linalg.norm(tangent)

        top_half = top_width / 2
        base_half = base_width / 2

        v0 = center_base - tangent * base_half
        v1 = center_base + tangent * base_half
        v2 = center_tip + tangent * top_half
        v3 = center_tip - tangent * top_half

        v0_up = np.append(v0, thickness)
        v1_up = np.append(v1, thickness)
        v2_up = np.append(v2, thickness)
        v3_up = np.append(v3, thickness)

        v0 = np.append(v0, 0)
        v1 = np.append(v1, 0)
        v2 = np.append(v2, 0)
        v3 = np.append(v3, 0)

        top_face = [v0, v1, v2, v3]
        bottom_face = [v0_up, v1_up, v2_up, v3_up]
        side1 = [v0, v0_up, v3_up, v3]
        side2 = [v1, v1_up, v2_up, v2]
        front = [v3, v2, v2_up, v3_up]

        ax.add_collection3d(Poly3DCollection([top_face], facecolors='lightgray', edgecolors='none',alpha=0.3))
        ax.add_collection3d(Poly3DCollection([bottom_face], facecolors='lightgray', edgecolors='none',alpha=0.3))
        ax.add_collection3d(Poly3DCollection([side1], facecolors='gray', edgecolors='k',alpha=0.3))
        ax.add_collection3d(Poly3DCollection([side2], facecolors='gray', edgecolors='k',alpha=0.3))
        ax.add_collection3d(Poly3DCollection([front], facecolors='dimgray', edgecolors='k',alpha=0.3))

    # Add central cylinder (solid disc)
    n_circle = 100
    theta = np.linspace(0, 2 * np.pi, n_circle)
    r_inner = 0.0
    r_outer = root_radius

    # Lower circle (z = 0)
    x_lower = r_outer * np.cos(theta)
    y_lower = r_outer * np.sin(theta)
    verts_lower = [[0, 0, 0]] + [[x, y, 0] for x, y in zip(x_lower, y_lower)]

    # Upper circle (z = thickness)
    x_upper = r_outer * np.cos(theta)
    y_upper = r_outer * np.sin(theta)
    verts_upper = [[0, 0, thickness]] + [[x, y, thickness] for x, y in zip(x_upper, y_upper)]

    # Side wall of the cylinder
    for i in range(n_circle - 1):
        x0, y0 = x_lower[i], y_lower[i]
        x1, y1 = x_lower[i + 1], y_lower[i + 1]
        ax.add_collection3d(Poly3DCollection(
            [[
                [x0, y0, 0],
                [x1, y1, 0],
                [x1, y1, thickness],
                [x0, y0, thickness]
            ]],
            facecolors='gray', edgecolors='none',alpha=0.3
        ))

    # Close the loop
    x0, y0 = x_lower[-1], y_lower[-1]
    x1, y1 = x_lower[0], y_lower[0]
    ax.add_collection3d(Poly3DCollection(
        [[
            [x0, y0, 0],
            [x1, y1, 0],
            [x1, y1, thickness],
            [x0, y0, thickness]
        ]],
        facecolors='gray', edgecolors='none',alpha=0.3
    ))

    # Create only the outer ring to avoid radial lines
    ring_lower = [v[:3] for v in verts_lower[1:]]  # exclude center point [0, 0, 0]
    ring_upper = [v[:3] for v in verts_upper[1:]]  # exclude center point [0, 0, height]

    ax.add_collection3d(Poly3DCollection([ring_lower], facecolors='gray', edgecolors='k', alpha=0.3, closed=True))
    ax.add_collection3d(Poly3DCollection([ring_upper], facecolors='gray', edgecolors='k', alpha=0.3, closed=True))

    # Add custom blue arrows for X, Y, Z axes (20% longer than outer_diameter)
    arrow_color = 'blue'

    # Z axis (vertical)
    ax.quiver(0, 0, 0, 0, 0, thickness *1.5, color=arrow_color, arrow_length_ratio=0.01)
    ax.text(0, 0, thickness * 1.05, 'Z', color=arrow_color)

    # X axis → radial, from center to first tooth base center (θ = 0), scaled
    theta0 = 0
    r_base = dimensions["root_diameter"] / 2
    x_target = r_base * np.cos(theta0) * 1.5
    y_target = r_base * np.sin(theta0) * 1.5
    ax.quiver(0, 0, 0, x_target, y_target, 0, color=arrow_color, arrow_length_ratio=0.01)
    ax.text(x_target * 1.02, y_target * 1.02, 0, 'X', color=arrow_color)

    # Y axis → tangent to the pitch at θ = 0, scaled
    x_tangent = -np.sin(theta0) * r_base * 1.5
    y_tangent =  np.cos(theta0) * r_base * 1.5
    ax.quiver(0, 0, 0, x_tangent, y_tangent, 0, color=arrow_color, arrow_length_ratio=0.01)
    ax.text(x_tangent * 1.02, y_tangent * 1.02, 0, 'Y', color=arrow_color)

    ax.set_box_aspect([1, 1, 0.3])
    ax.view_init(elev=30, azim=45)
    ax.set_axis_off()
    plt.tight_layout()

    # Salva e mostra immagine
    tmpfile = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    plt.savefig(tmpfile.name)
    plt.close(fig)

    display(Image(tmpfile.name))
    time.sleep(0.5)

# Example gear dimensions
dimensions = {
    "number_of_teeth": 20,
    "outer_diameter": 100.0,           # mm
    "root_diameter": 80.0,             # mm
    "tooth_top_width": 6.0,            # mm
    "tooth_base_width": 10.0,          # mm
    "tooth_height": (100 - 80) / 2,    # mm
    "gear_thickness": 10.0,            # mm
    "tooth_clearance_angle": 0.0       # degrees (not used in current version)
}

plot_gear_3d(dimensions)

# Text Input

In [ ]:
def collect_part_info():
    SUPPORTED_GEOMETRIES = ["Plate", "Cylinder", "Gear", "Flange", "Bracket", "Block"]
    print("Fill in the following information about the part to be machined:\n")

    material = input("Material: ")
    print("Select the type of geometry:")
    for i, g in enumerate(SUPPORTED_GEOMETRIES):
        print(f"{i+1}. {g}")

    choice = int(input("Enter the corresponding number: ")) - 1
    geometry = SUPPORTED_GEOMETRIES[choice]
    dimensions = ask_dimensions(geometry)
    origin = define_reference_origin(geometry, dimensions)
    features = collect_features()
    tolerance_val = input("Required tolerance ± ... mm → Enter only the value: ")
    tolerance = f"±{tolerance_val} mm"

    finish_val = input("Surface finish Ra < ... µm → Enter only the value: ")
    finish = f"Ra < {finish_val} µm"

    part_info = {
        "geometry": geometry,
        "geometry_parameters": dimensions,
        "material": material,
        "reference_origin": origin,
        "features": features,
        "tolerance": tolerance,
        "surface_finish": finish
    }

    print("\nCollected information:")
    for k, v in part_info.items():
        print(f"- {k}: {v}")

    return part_info

def ask_dimensions(geometry):
    print(f"\nEnter the dimensional parameters for geometry: {geometry}\n")
    dimensions = {}

    if geometry == "Plate":
        dimensions["length_mm"] = float(input("Length (mm): "))
        dimensions["width_mm"] = float(input("Width (mm): "))
        dimensions["thickness_mm"] = float(input("Thickness (mm): "))

    elif geometry == "Cylinder":
        dimensions["diameter_mm"] = float(input("Diameter (mm): "))
        dimensions["height_mm"] = float(input("Height (mm): "))

    elif geometry == "Gear":
        print("\n--- Define gear parameters ---")
        # Core parameters for trapezoidal gear shape
        dimensions["number_of_teeth"] = int(input("Number of teeth: "))
        dimensions["module"] = float(input("Module (mm): "))
        dimensions["pressure_angle"] = float(input("Pressure angle (degrees), typically 20: "))

        # Derived involute geometry
        dimensions["pitch_diameter"] = dimensions["module"] * dimensions["number_of_teeth"]
        dimensions["addendum"] = dimensions["module"]
        dimensions["dedendum"] = 1.25 * dimensions["module"]
        dimensions["outer_diameter"] = dimensions["pitch_diameter"] + 2 * dimensions["addendum"]
        dimensions["root_diameter"] = dimensions["pitch_diameter"] - 2 * dimensions["dedendum"]

        # Useful for trapezoidal plotting
        dimensions["tooth_height"] = dimensions["outer_diameter"] / 2 - dimensions["root_diameter"] / 2
        dimensions["tooth_top_width"] = float(input("Tooth top width (Width of the trapezoid at the tip): "))
        dimensions["tooth_base_width"] = float(input("Tooth base width (Width of the trapezoid at the base): "))

        # 3D extrusion
        dimensions["gear_thickness"] = float(input("Gear thickness (Extrusion depth in mm): "))

        # Optional manufacturing tolerance
        dimensions["tooth_clearance_angle"] = float(input("Tooth clearance angle (deg, optional, set 0 if not needed): "))
    elif geometry == "Flange":
        dimensions["outer_diameter_mm"] = float(input("Outer diameter (mm): "))
        dimensions["thickness_mm"] = float(input("Thickness (mm): "))
        dimensions["number_of_holes"] = int(input("Number of holes: "))

    elif geometry == "Bracket":
        dimensions["width_mm"] = float(input("Width (mm): "))
        dimensions["height_mm"] = float(input("Height (mm): "))
        dimensions["thickness_mm"] = float(input("Thickness (mm): "))
        dimensions["bend_angle_deg"] = float(input("Bend angle (°): "))

    elif geometry == "Block":
        dimensions["length_mm"] = float(input("Length (mm): "))
        dimensions["width_mm"] = float(input("Width (mm): "))
        dimensions["height_mm"] = float(input("Height (mm): "))

    else:
        print("Unrecognized geometry. No parameters required.")

    return dimensions

def define_reference_origin(geometry, dimensions):
    print("\n--- DEFINITION OF THE ORIGIN AND AXES OF THE REFERENCE SYSTEM (WCS) ---")
    plot_reference_system(geometry, dimensions)

    if geometry in ["Plate", "Block"]:
        origin_descr = "Center of the mid-plane"
        orientation_descr = "X along the length direction, Y along the width direction, Z along thickness direction"
    elif geometry in ["Cylinder"]:
        origin_descr = "Center of the cylinder on the lower surface"
        orientation_descr = "X radial, Y radial, Z along the cylinder axis (upwards)"
    elif geometry in [ "Gear", "Flange"]:
        origin_descr = "Center of the lower surface of the Gear"
        orientation_descr = "X radial, Y radial, Z along the cylinder axis (upwards)"
    elif geometry == "Bracket":
        origin_descr = "Approximate center of the bent plane"
        orientation_descr = "X along the base, Y along height, Z through thickness"
    else:
        origin_descr = "Estimated geometric center of the part"
        orientation_descr = "X, Y, Z oriented according to neutral geometry (Cartesian)"

    print(f"\nSuggested origin: **{origin_descr}** → considered as (0, 0, 0)")
    print(f"Default axis orientation: {orientation_descr}")

    choice = input("\nDo you want to use this origin and orientation? (y = yes, n = no): ").strip().lower()

    if choice == 'y':
        return {
            "origin_mm": [0.0, 0.0, 0.0],
            "origin_description": origin_descr,
            "axis_orientation": orientation_descr,
            "rotation_deg": [0.0, 0.0, 0.0]
        }

    elif choice == 'n':
        print("\nEnter an offset from the suggested point (in mm):")
        x = float(input("Offset X: "))
        y = float(input("Offset Y: "))
        z = float(input("Offset Z: "))
        new_origin = [round(x, 2), round(y, 2), round(z, 2)]

        rotation = [0.0, 0.0, 0.0]
        rot_choice = input("Do you also want to rotate the reference system? (y/n): ").strip().lower()
        if rot_choice == 'y':
            print("Enter rotation angles around the axes (in degrees):")
            rx = float(input("Rotation around X (°): "))
            ry = float(input("Rotation around Y (°): "))
            rz = float(input("Rotation around Z (°): "))
            rotation = [round(rx, 2), round(ry, 2), round(rz, 2)]

        return {
            "origin_mm": new_origin,
            "origin_description": f"Offset origin relative to: {origin_descr}",
            "axis_orientation": orientation_descr,
            "rotation_deg": rotation
        }

    else:
        print("Invalid answer. Using default values.")
        return {
            "origin_mm": [0.0, 0.0, 0.0],
            "origin_description": origin_descr,
            "axis_orientation": orientation_descr,
            "rotation_deg": [0.0, 0.0, 0.0]
        }

def define_machining_direction():
    print("\nDefinition of machining direction:")
    choice = input("Do you want to align machining to one of the main X/Y/Z axes? (y/n): ").strip().lower()

    if choice == 'y':
        axis = input("Specify the machining axis (X / Y / Z): ").strip().upper()
        if axis in ["X", "Y", "Z"]:
            return {"type": "main_axis", "direction": axis}
        else:
            print("Invalid axis. Using Z axis by default.")
            return {"type": "main_axis", "direction": "Z"}

    elif choice == 'n':
        print("Specify the inclined machining direction:")
        angle_xy = float(input("Angle in the XY plane w.r.t. X axis (degrees, clockwise): "))
        angle_z = float(input("Inclination angle w.r.t. Z axis (degrees, clockwise): "))
        return {
            "type": "inclined_direction",
            "angle_xy_deg": round(angle_xy, 2),
            "angle_z_deg": round(angle_z, 2)
        }

    else:
        print("Invalid answer. Using Z axis by default.")
        return {"type": "main_axis", "direction": "Z"}

def collect_features():
    print("\n--- GEOMETRIC FEATURE COLLECTION ---")
    features = []
    count = 1

    FEATURE_TYPES = {
        "1": "Through hole",
        "2": "Blind hole",
        "3": "Rectangular pocket",
        "4": "Circular pocket",
        "5": "Slot",
        "6": "Chamfer",
        "7": "Thread",
        "8": "Other (free description)"
    }

    while True:
        choice = input(f"\nDo you want to add feature #{count}? (y/n): ").strip().lower()
        if choice == 'n':
            break
        elif choice != 'y':
            print("Invalid response. Please enter 'y' or 'n'.")
            continue

        print("\nSelect the type of feature:")
        for code, name in FEATURE_TYPES.items():
            print(f"{code}. {name}")
        ftype = input("Number corresponding to the feature: ").strip()

        if ftype not in FEATURE_TYPES:
            print("Invalid feature type.")
            continue

        feature = {"type": FEATURE_TYPES[ftype]}

        if ftype == "1":  # Through hole
            feature["diameter_mm"] = float(input("Diameter (mm): "))
            x = float(input("Center X position (mm): "))
            y = float(input("Center Y position (mm): "))
            z = float(input("Center Z position (mm): "))
            feature["position_mm"] = [x, y, z]
            feature["machining_direction"] = define_machining_direction()

        elif ftype == "2":  # Blind hole
            feature["diameter_mm"] = float(input("Diameter (mm): "))
            feature["depth_mm"] = float(input("Depth (mm): "))
            x = float(input("X position (mm): "))
            y = float(input("Y position (mm): "))
            z = float(input("Z position (mm): "))
            feature["position_mm"] = [x, y, z]
            feature["machining_direction"] = define_machining_direction()

        elif ftype == "3":  # Rectangular pocket
            feature["length_mm"] = float(input("Length (mm): "))
            feature["width_mm"] = float(input("Width (mm): "))
            feature["depth_mm"] = float(input("Depth (mm): "))
            x = float(input("X position (mm): "))
            y = float(input("Y position (mm): "))
            feature["position_mm"] = [x, y]

        elif ftype == "4":  # Circular pocket
            feature["diameter_mm"] = float(input("Diameter (mm): "))
            feature["depth_mm"] = float(input("Depth (mm): "))
            x = float(input("X position (mm): "))
            y = float(input("Y position (mm): "))
            feature["position_mm"] = [x, y]

        elif ftype == "5":  # Slot
            feature["length_mm"] = float(input("Length (mm): "))
            feature["width_mm"] = float(input("Width (mm): "))
            x = float(input("X position (mm): "))
            y = float(input("Y position (mm): "))
            feature["position_mm"] = [x, y]

        elif ftype == "6":  # Chamfer / Fillet
            feature["radius_mm"] = float(input("Radius (mm): "))
            feature["edge"] = input("Edge to apply the feature (e.g., top edge, right edge...): ").strip()

        elif ftype == "7":  # Thread
            feature["nominal_diameter_mm"] = float(input("Nominal diameter (mm): "))
            feature["pitch_mm"] = float(input("Pitch (mm): "))
            feature["depth_mm"] = float(input("Depth (mm): "))
            x = float(input("X position (mm): "))
            y = float(input("Y position (mm): "))
            feature["position_mm"] = [x, y]

        elif ftype == "8":  # Other
            desc = input("Free description of the feature: ").strip()
            if desc:
                feature["description"] = desc
            else:
                print("Empty description not valid. Feature ignored.")
                continue

        features.append(feature)
        count += 1

    return features

def plot_reference_system(geometry, dimensions):

    if geometry == "Gear":
        plot_gear_3d(dimensions)
    elif geometry == "Cylinder":
        plot_cylinder_3d(dimensions)
    elif geometry == "Plate":
        plot_plate_3d(dimensions)
    # Aggiungi altri casi se servono


In [ ]:
53# Esegui la funzione per raccogliere input
part_info = collect_part_info()

# Esegui la funzione per raccogliere input
print("-------------- PART DESCRIPTION -----------------------")
for k, v in part_info.items():
        print(f"- {k}: {v}")

In [34]:
# PART INFO DEFAULT text
part_info = {'geometry': 'Plate',
             'geometry_parameters': {'length_mm': 200.0, 'width_mm': 100.0, 'thickness_mm': 20.0},
             'material': 'aluminum',
             'reference_origin': {'origin_mm': [0.0, 0.0, 0.0], 'origin_description': 'Center of the mid-plane', 'axis_orientation': 'X to the right, Y in depth, Z upwards', 'rotation_deg': [0.0, 0.0, 0.0]},
             'features': [{'type': 'Rectangular pocket', 'length_mm': 10.0, 'width_mm': 10.0, 'depth_mm': 10.0, 'position_mm': [50.0, 0.0]},
                          {'type': 'Through hole', 'diameter_mm': 5.0, 'position_mm': [50.0, 0.0], 'machining_direction': {'type': 'main_axis', 'direction': 'Z'}},
                          {'type': 'Through hole', 'diameter_mm': 5.0, 'position_mm': [-50.0, 0.0], 'machining_direction': {'type': 'main_axis', 'direction': 'Z'}},
                          {'type': 'Rectangular pocket', 'length_mm': 10.0, 'width_mm': 60.0, 'depth_mm': 5.0, 'position_mm': [-20.0, 0.0]},
                          {'type': 'Chamfer', 'radius_mm': 6.0, 'edge': 'top edge'},
                          {'type': 'Fillet', 'radius_mm': 5.0, 'edge': 'right edge'}],
             'tolerance': '±3 mm',
             'surface_finish': 'Ra < 40 µm'}

# LLM Call

**Environment and prompt setup**

In [ ]:
# Installation
!pip install tiktoken
!pip install jsonschema


In [35]:
import openai
import tiktoken
import json
import re
from jsonschema import validate, ValidationError
# Get key
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_API')
openai.api_key = openai_api_key
ENGINE =   "gpt-4o-mini"


# ------------------------------ OPERATIONS -------------------------------------
operation_constraints = {
    "Drilling (Through Hole)": {
        "rpm_max": 6000,
        "max_depth": 100,
        "max_tool_diameter": 25,
        "note": "High torque required for deep holes"
    },
    "Drilling (Blind Hole)": {
        "rpm_max": 6000,
        "max_depth": 80,
        "max_tool_diameter": 20,
        "note": "Careful chip evacuation needed"
    },
    "Face Milling": {
        "rpm_max": 8000,
        "max_depth": 5,
        "max_tool_diameter": 50,
        "note": "Large diameter end mills"
    },
    "Slot Milling": {
        "rpm_max": 7000,
        "max_depth": 20,
        "max_tool_diameter": 16,
        "note": "Requires stable fixturing"
    },
    "Contour Milling": {
        "rpm_max": 7500,
        "max_depth": 10,
        "max_tool_diameter": 20,
        "note": "High precision finish"
    },
    "Threading": {
        "rpm_max": 3000,
        "max_feed": 300,
        "max_tool_diameter": 12,
        "note": "Use correct tap cycle"
    },
    "Boring": {
        "rpm_max": 5000,
        "max_depth": 120,
        "max_tool_diameter": 40,
        "note": "Requires boring head attachment"
    },
    "Reaming": {
        "rpm_max": 4000,
        "max_depth": 50,
        "max_tool_diameter": 18,
        "note": "Use coolant for finish"
    },
    "Chamfering": {
        "rpm_max": 10000,
        "max_depth": 2,
        "max_tool_diameter": 10,
        "note": "45-degree chamfer tools"
    },
    "Tapping": {
        "rpm_max": 2500,
        "max_depth": 30,
        "max_tool_diameter": 12,
        "note": "Synchronous spindle required"
    }
}

# ----------------------------------- TOOLS ---------------------------------------------------
available_tools = [
    {"name": "Twist Drill", "diameter_mm": list(range(2, 26))},
    {"name": "End Mill", "diameter_mm": list(range(1, 51))},
    {"name": "Chamfer Mill", "diameter_mm": [5, 10, 15, 20]},
    {"name": "Ball Nose End Mill", "diameter_mm": [3, 6, 12, 16]},
    {"name": "Tapered End Mill", "diameter_mm": [5, 10, 20]},
    {"name": "Reamer", "diameter_mm": [6, 8, 10, 12, 16]},
    {"name": "Boring Head", "diameter_mm": [20, 30, 40]},
    {"name": "Thread Tap", "diameter_mm": [3, 4, 5, 6, 8, 10, 12]},
    {"name": "Countersink", "diameter_mm": [8, 10, 12]},
    {"name": "Face Mill Cutter", "diameter_mm": [50, 63, 80]}
]

# Operations & tools definitions

system_message1 = (
    "You are a CNC expert. Your job is to analyze a part description and identify the required machining operations.\n"
    "You must:\n"
    "1. List ONLY the operations needed to manufacture the part.\n"
    "2. For each operation, select ONE suitable tool from the available tools list.\n"
    "3. Follow machine constraints: allowed operations, tools, RPM, and depth limits.\n\n"
    "Available operations:\n"
    + "\n".join(f"- {op}: {c['note']} (RPM ≤{c['rpm_max']}, Depth ≤{c.get('max_depth','N/A')}, Tool Ø ≤{c.get('max_tool_diameter','N/A')})"
                for op, c in operation_constraints.items())
    + "\n\nAvailable tools:\n"
    + "\n".join(f"- {t['name']}, diameters: {t['diameter_mm']}" for t in available_tools)
    + "\n\nRespond ONLY in valid JSON format with this structure:\n"
    """{
     "Operations": [
        {
          "Operation": "...",
          "Tool": "...",
          "Position": "...",
          "RPM": "...",
          "Depth": "...",
          "Notes": "..."
        },
        {....},
        .
        .
        .
      ]
    }"""
)

def build_prompt1(data):
    return f"\nHere is the part description:\n{json.dumps(data, indent=2)}"


# Process plan & setup plan definitions

system_message2 = (
    "You are a CNC process planner. Given a part description and a list of operations/tools, your job is to:\n"
    "1. Define a setup plan: number of setups, orientation, and fixturing for each setup.\n"
    "2. Propose an optimal sequence of operations (process plan) across the setups.\n"
    "3. Include RPM, feedrate and depth, considering material constraints.\n\n"
    "Output format (valid JSON only):\n"
    """{
      "Setup plan": [
        {
          "Setup number": 1,
          "Orientation": "...",
          "Fixturing": "...",
          "Notes": "..."
        },
        ...
      ],
      "Process plan": [
        {
          "Step number": 1,
          "Setup number": 1,
          "Operation": "...",
          "Tool": "...",
          "RPM": ...,
          "Feed": ...,
          "Depth": ...,
          "Notes": "..."
        },
        ...
      ]
    }"""
)

def build_prompt2(part_info, operations_output):
    return (
        f"Part description:\n{json.dumps(part_info, indent=2)}\n\n"
        f"Operations and selected tools:\n{json.dumps(operations_output['Operations'], indent=2)}"
    )

# Call API
def call_api(message):
    try:
        response = openai.chat.completions.create(
            model=ENGINE,
            messages=[{"role": "user", "content": message}],
            temperature=0.8,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API call error: {e}")
        return None

def clean_llm_json_response(text: str) -> str:
    """
    Pulisce l'output dell'LLM rimuovendo delimitatori ```json e ```
    Rende il testo compatibile con json.loads()
    """
    lines = text.strip().splitlines()
    # Rimuovi tutte le righe che iniziano con ```
    lines = [line for line in lines if not line.strip().startswith("```")]
    return "\n".join(lines).strip()




**Validation**

In [36]:
from typing import Tuple
from collections import defaultdict

# Link between operations and tools
operation_to_tool_type = {
    "Drilling (Through Hole)": {"Twist Drill"},
    "Drilling (Blind Hole)": {"Twist Drill"},
    "Face Milling": {"Face Mill Cutter", "End Mill"},
    "Slot Milling": {"End Mill", "Ball Nose End Mill"},
    "Contour Milling": {"End Mill", "Ball Nose End Mill", "Tapered End Mill"},
    "Chamfering": {"Chamfer Mill", "Countersink"},
    "Tapping": {"Thread Tap"},
    "Threading": {"Thread Tap"},
    "Boring": {"Boring Head"},
    "Reaming": {"Reamer"},
}


# Each tuple means: the first operation (A) must appear
# BEFORE every occurrence of the second operation (B) within the same Setup.
precedence_rules = [
    # Hole finishing chain
    ("Drilling",  "Boring"),      # pilot drill → bore to size
    ("Drilling",  "Reaming"),     # drill → ream for tolerance
    ("Drilling",  "Tapping"),     # drill → tap internal thread
    ("Drilling",  "Threading"),   # drill → single-point thread

    ("Boring",    "Reaming"),     # bore → ream (fine)

    # Planar rough/finish chain
    ("Face Milling",   "Contour Milling"),  # rough face → finish contour
    ("Face Milling",   "Slot Milling"),     # rough face → cut slot
    ("Face Milling",   "Chamfering"),       # rough face → chamfer edges
    ("Slot Milling",   "Chamfering"),       # cut slot → chamfer slot edges

    # Always-last rule
    ("Any",      "Chamfering"),   # chamfer is one of the final ops
]

def op_category(op_name: str) -> str:
    """
    Converts full operation label to a category keyword
    used in the precedence rules above.
    """
    if "Drilling" in op_name:  return "Drilling"
    if "Boring"   in op_name:  return "Boring"
    if "Reaming"  in op_name:  return "Reaming"
    if "Tapping"  in op_name:  return "Tapping"
    if "Threading" in op_name: return "Threading"
    if "Face Milling" in op_name:   return "Face Milling"
    if "Slot Milling" in op_name:   return "Slot Milling"
    if "Contour Milling" in op_name: return "Contour Milling"
    if "Chamfering" in op_name:     return "Chamfering"
    # fallback: return the name itself
    return op_name

def validate_precedence(process_plan: List[Dict]) -> Tuple[bool, str]:
    """
    Checks precedence rules within each Setup number.
    Returns (is_valid, error_string)
    """
    # Group steps by Setup number
    steps_by_setup = defaultdict(list)
    for step in process_plan:
        steps_by_setup[step["Setup number"]].append(step)

    errors = []

    # Walk each setup independently
    for setup_no, steps in steps_by_setup.items():
        # Ensure natural order by Step number
        steps_sorted = sorted(steps, key=lambda s: s["Step number"])
        seen = set()                               # categories already executed

        for step in steps_sorted:
            idx = step["Step number"]
            cat = op_category(step["Operation"])

            # Check every rule whose B matches current category
            for A, B in precedence_rules:
                if (B == "Any") or (B == cat):
                    # For 'Any' rule we test only if B is 'Chamfering'
                    if B == "Chamfering" and cat != "Chamfering":
                        continue

                    # -------- precedence violation cases --------
                    if A != "Any" and A not in seen:
                        errors.append(
                            f"Setup {setup_no}, Step {idx}: '{cat}' "
                            f"appears before any '{A}'."
                        )
                    if A == "Any" and cat == "Chamfering" and idx != steps_sorted[-1]["Step number"]:
                        errors.append(
                            f"Setup {setup_no}, Step {idx}: Chamfering should be last in its setup."
                        )

            # Record that this category is now done
            seen.add(cat)

    if errors:
        return False, "\n".join(errors)

    return True, ""




def validate_call1(response_1: str) -> Tuple[bool, str]:
    try:
        data = json.loads(response_1)
        operations = data.get("Operations", [])
    except json.JSONDecodeError as e:
        return False, f"❌ JSON parsing error: {e}"

    errors = []

    # Valid operations from constraints
    valid_operations = set(operation_constraints.keys())

    # Tool dictionary: tool name → set of available diameters
    tool_dict = {tool["name"]: set(tool["diameter_mm"]) for tool in available_tools}

    for idx, op in enumerate(operations, 1):
        op_name = op.get("Operation", "").strip()
        tool_str = op.get("Tool", "").strip()

        # ----- Check 1: Operation exists -----
        if op_name not in valid_operations:
            errors.append(f"❌ Step #{idx}: '{op_name}' is not a supported operation.")
            continue  # Skip further checks for this operation

        # ----- Check 2: Tool format -----
        match = re.match(r"(.+?),\s*diameter[:\s]*([0-9]+)", tool_str)
        if not match:
            errors.append(f"❌ Step #{idx}: Tool format invalid → '{tool_str}' (expected 'Tool Name, diameter: N')")
            continue

        tool_name = match.group(1).strip()
        try:
            tool_diameter = int(match.group(2))
        except ValueError:
            errors.append(f"❌ Step #{idx}: Tool diameter is not a valid integer → '{tool_str}'")
            continue

        # ----- Check 3: Tool exists -----
        if tool_name not in tool_dict:
            errors.append(f"❌ Step #{idx}: Tool '{tool_name}' is not in available tool list.")
            continue

        # ----- Check 4: Diameter is allowed -----
        if tool_diameter not in tool_dict[tool_name]:
            valid_sizes = ", ".join(map(str, sorted(tool_dict[tool_name])))
            errors.append(f"❌ Step #{idx}: Tool '{tool_name}' does not support Ø{tool_diameter}. Valid: [{valid_sizes}]")

        # ----- Check 5: Tool type matches operation type -----
        valid_tools_for_op = operation_to_tool_type.get(op_name, set())
        if tool_name not in valid_tools_for_op:
            valid_tool_list = ", ".join(valid_tools_for_op) if valid_tools_for_op else "N/A"
            errors.append(
                f"❌ Step #{idx}: Tool '{tool_name}' is not valid for operation '{op_name}'. Expected one of: [{valid_tool_list}]"
            )

    # ----- Return result -----
    if errors:
        error_msg = "\n".join(errors)
        return False, error_msg

    return True, ""


def validate_call2(response_2: str) -> Tuple[bool, str]:
    try:
        data = json.loads(response_2)
        process_plan = data.get("Process plan", [])
    except json.JSONDecodeError as e:
        return False, f"❌ JSON parsing error: {e}"

    errors = []

    for step in process_plan:
        step_num = step.get("Step number", "?")
        operation = step.get("Operation", "").strip()
        rpm = step.get("RPM", "")
        depth = step.get("Depth", "")
        tool_str = step.get("Tool", "").strip()

        # --------- Check 1: operation must exist ---------
        if operation not in operation_constraints:
            errors.append(f"❌ Step #{step_num}: Unknown operation '{operation}'")
            continue  # Skip parameter checks

        op_constraints = operation_constraints[operation]

        # --------- Check 2: RPM limit ---------
        if "rpm_max" in op_constraints:
            try:
                rpm_val = float(rpm)
                if rpm_val > op_constraints["rpm_max"]:
                    errors.append(
                        f"❌ Step #{step_num}: RPM {rpm_val} exceeds max RPM ({op_constraints['rpm_max']}) for '{operation}'"
                    )
            except ValueError:
                errors.append(f"❌ Step #{step_num}: RPM '{rpm}' is not a valid number")

        # --------- Check 3: Depth limit ---------
        if "max_depth" in op_constraints:
            try:
                depth_val = float(depth)
                if depth_val > op_constraints["max_depth"]:
                    errors.append(
                        f"❌ Step #{step_num}: Depth {depth_val} exceeds max depth ({op_constraints['max_depth']}) for '{operation}'"
                    )
            except ValueError:
                errors.append(f"❌ Step #{step_num}: Depth '{depth}' is not a valid number")

        # --------- Check 4: Tool diameter limit ---------
        if "max_tool_diameter" in op_constraints:
            match = re.search(r"diameter[:\s]*([0-9]+)", tool_str)
            if match:
                try:
                    tool_diameter = int(match.group(1))
                    if tool_diameter > op_constraints["max_tool_diameter"]:
                        errors.append(
                            f"❌ Step #{step_num}: Tool Ø{tool_diameter} exceeds max Ø{op_constraints['max_tool_diameter']} for '{operation}'"
                        )
                except ValueError:
                    errors.append(f"❌ Step #{step_num}: Invalid tool diameter in '{tool_str}'")
            else:
                errors.append(f"❌ Step #{step_num}: Tool diameter not found in '{tool_str}'")

        # --------- Check Order Consistency ---------
        seq_ok, seq_errors = validate_precedence(process_plan)
        if not seq_ok:
            errors.append(seq_errors)

    if errors:
        return False, "\n".join(errors)

    return True, ""

def validate_loop(response_1: str, response_2: str) -> bool:
    try:
        data1 = json.loads(response_1)
    except json.JSONDecodeError as e:
        print(f"❌ Call 1 JSON error: {e}")
        return False

    try:
        data2 = json.loads(response_2)
    except json.JSONDecodeError as e:
        print(f"❌ Call 2 JSON error: {e}")
        return False

    # --- Check response_1 minimal structure ---
    if "Operations" not in data1 or not isinstance(data1["Operations"], list):
        print("❌ Call 1 is missing 'Operations' or it's not a list.")
        return False

    if not data1["Operations"]:
        print("❌ Call 1: 'Operations' list is empty.")
        return False

    # --- Check response_2 minimal structure ---
    if "Setup plan" not in data2 or not isinstance(data2["Setup plan"], list):
        print("❌ Call 2 is missing 'Setup plan' or it's not a list.")
        return False

    if "Process plan" not in data2 or not isinstance(data2["Process plan"], list):
        print("❌ Call 2 is missing 'Process plan' or it's not a list.")
        return False

    if not data2["Process plan"]:
        print("❌ Call 2: 'Process plan' list is empty.")
        return False

    # --- Passed fallback checks ---
    print("✅ validate_loop: minimal required structure present in both responses.")
    return True


**LLM call loop**

In [37]:
is_loop_correct = False
is_call1_correct = False
is_call2_correct = False
count_loop = 0
count_call1 = 0
count_call2 = 0
max_loop = 3
max_call1 = 3
max_call2 = 3

# Main loop: repeats the entire process until both calls are valid or max attempts are reached
while not is_loop_correct and count_loop < max_loop:
    count_loop += 1
    print(f"🔁 Loop {count_loop}/{max_loop}")

    # ------------------- FIRST LLM CALL: OPERATIONS and TOOLS SELECTION ----------------------
    while not is_call1_correct and count_call1 < max_call1:
        count_call1 += 1
        print(f"📞 Call 1 (Setup plan) {count_call1}/{max_call1}")

        # Compose the full message for the LLM (system message + user prompt)
        message1 = system_message1 + build_prompt1(part_info)
        # Call the LLM API and receive the raw response
        response_1 = call_api(message1)
        # Clean the LLM response to remove markdown formatting like ```json
        response_1 = clean_llm_json_response(response_1)

        # Validate the response
        is_call1_correct, error_1 = validate_call1(response_1)
        # If max attempts are reached without success, stop trying
        if not is_call1_correct and count_call1 == max_call1:
            print("❌ Call 1 failed after maximum attempts. Exiting.")
            print(error_1)
            break
        elif not is_call1_correct:
            print("❌ Call 1 failed. Retrying...")
            message1 = system_message1 + build_prompt1(part_info) + error_1
            print(error_1)
        else:
            print("✅ Call 1 ok.")

    # ---------------------- SECOND LLM CALL: SETUP and PROCESS PLAN ----------------------
    while not is_call2_correct and count_call2 < max_call2:
        count_call2 += 1
        print(f"📞 Call 2 (Process plan) {count_call2}/{max_call2}")

        # Try to parse response_1
        try:
            parsed_response_1 = json.loads(response_1)
        except json.JSONDecodeError as e:
            print("❌ JSON decoding error in response_1:", e)
            break

        # Compose the full message for the second call, using the result of call 1
        message2 = system_message2 + build_prompt2(part_info, parsed_response_1)

        # Call the LLM API
        response_2 = call_api(message2)

        # Clean markdown formatting from response
        response_2 = clean_llm_json_response(response_2)

        # Validate the second response
        is_call2_correct, error_2 = validate_call2(response_2)

        # If max attempts are reached without success, stop trying
        if not is_call2_correct and count_call2 == max_call2:
            print("❌ Call 2 failed after maximum attempts. Exiting.")
            print(error_2)
            break
        elif not is_call2_correct:
            print("❌ Call 2 failed. Retrying...")
            message2 = system_message2 + build_prompt2(part_info,json.loads(response_1)) + error_2
            print(error_2)
        else:
            print("✅ Call 2 ok.")

    # ---------------------- FINAL VALIDATION: COMPLETE PLAN ----------------------
    if is_call1_correct and is_call2_correct:
        # Optional final validation for the combined output
        is_loop_correct = validate_loop(response_1, response_2)




# # Call 1
# # print(system_message1)
# # print(build_prompt1(part_info))
# message1 = system_message1 + build_prompt1(part_info)
# response_1 = call_api(message1)

# response_1 = clean_llm_json_response(response_1)
# print(response_1)
# print("\n")
# print("\n")
# # # Call 2
# # print(system_message2)
# # print(build_prompt2(part_info,json.loads(response_1)))
# message2 = system_message2 + build_prompt2(part_info, json.loads(response_1))
# response_2 = call_api(message2)
# response_2 = clean_llm_json_response(response_2)
# print(response_2)

🔁 Loop 1/3
📞 Call 1 (Setup plan) 1/3
❌ Call 1 failed. Retrying...
❌ Step #1: Tool format invalid → 'End Mill, Ø 10' (expected 'Tool Name, diameter: N')
❌ Step #2: Tool format invalid → 'Twist Drill, Ø 5' (expected 'Tool Name, diameter: N')
❌ Step #3: Tool format invalid → 'Twist Drill, Ø 5' (expected 'Tool Name, diameter: N')
❌ Step #4: Tool format invalid → 'End Mill, Ø 10' (expected 'Tool Name, diameter: N')
❌ Step #5: Tool format invalid → 'Chamfer Mill, Ø 10' (expected 'Tool Name, diameter: N')
❌ Step #6: Tool format invalid → 'Ball Nose End Mill, Ø 6' (expected 'Tool Name, diameter: N')
📞 Call 1 (Setup plan) 2/3
❌ Call 1 failed. Retrying...
❌ Step #1: Tool format invalid → 'End Mill, Ø 10' (expected 'Tool Name, diameter: N')
❌ Step #2: Tool format invalid → 'Twist Drill, Ø 5' (expected 'Tool Name, diameter: N')
❌ Step #3: Tool format invalid → 'Twist Drill, Ø 5' (expected 'Tool Name, diameter: N')
❌ Step #4: Tool format invalid → 'End Mill, Ø 10' (expected 'Tool Name, diameter: N'

# Post-Processing

In [ ]:
import pandas as pd
from typing import List, Dict


def parse_full_plan_json(text: str) -> List[Dict]:
    """
    Dato un JSON come stringa, restituisce una lista ordinata di dizionari:
    - Setup #1
    - Tutti gli step associati a Setup #1
    - Setup #2
    - Tutti gli step associati a Setup #2
    - ...
    """
    data = json.loads(text)

    setup_plan = data.get("Setup plan", [])
    process_plan = data.get("Process plan", [])

    result: List[Dict] = []

    for setup in sorted(setup_plan, key=lambda x: x["Setup number"]):
        # Aggiungi prima il setup
        setup_entry = {
            "Type": "Setup",
            **setup
        }
        result.append(setup_entry)

        # Aggiungi tutti gli step associati a questo setup
        for step in sorted(process_plan, key=lambda x: x["Step number"]):
            if step["Setup number"] == setup["Setup number"]:
                step_entry = {
                    "Type": "Step",
                    **step
                }
                result.append(step_entry)

    return result

def plan_to_dataframe(plan_list: List[Dict]) -> pd.DataFrame:

    df = pd.DataFrame(plan_list)
    df.fillna("-", inplace=True)  # Rimpiazza NaN con "-"
    return df



# Esempio di utilizzo:
plan_list = parse_full_plan_json(response_2)  # `response` è la stringa JSON dell'LLM
df_steps = plan_to_dataframe(plan_list)
df_steps  # Per notebook Jupyter, mostra la tabella

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject